In [21]:
import cPickle as pickle
import numpy as np

with open('gvars_g1.pkl', 'rb') as f:
    g1_vars = pickle.load(f)
    
with open('gvars_g2.pkl', 'rb') as f:
    g2_vars = pickle.load(f)
    
with open('snapshot_z.pkl', 'rb') as f:
    snapshot_z = pickle.load(f)
    
print snapshot_z.shape

(8, 512)


In [12]:
g1n_to_var = {}
for (g1n, g1v), (g2n, g2v) in zip(g1_vars, g2_vars):
    g1n_to_var[g1n] = g1v
    print g1n
    print g1v.shape

G1a.W
(512, 512, 16)
G1aS.b
(512,)
G1b.W
(512, 512, 9)
G1bS.b
(512,)
G2a.W
(512, 512, 9)
G2aS.b
(512,)
G2b.W
(512, 512, 9)
G2bS.b
(512,)
G3a.W
(512, 512, 9)
G3aS.b
(512,)
G3b.W
(512, 512, 9)
G3bS.b
(512,)
G4a.W
(512, 512, 9)
G4aS.b
(512,)
G4b.W
(512, 512, 9)
G4bS.b
(512,)
G5a.W
(256, 512, 9)
G5aS.b
(256,)
G5b.W
(256, 256, 9)
G5bS.b
(256,)
G6a.W
(128, 256, 9)
G6aS.b
(128,)
G6b.W
(128, 128, 9)
G6bS.b
(128,)
Glod0.W
(128, 1)
Glod0S.b
(1,)
Glod1.W
(256, 1)
Glod1S.b
(1,)
Glod2.W
(512, 1)
Glod2S.b
(1,)
Glod3.W
(512, 1)
Glod3S.b
(1,)
Glod4.W
(512, 1)
Glod4S.b
(1,)
Glod5.W
(512, 1)
Glod5S.b
(1,)


In [35]:
import tensorflow as tf

def pixel_norm(x):
    return x / tf.sqrt(tf.reduce_mean(tf.square(x), axis=1, keep_dims=True) + 1e-8)

def w_scale(x):
    return x

tf.reset_default_graph()

z = tf.placeholder(tf.float32, [None, 512])

z_norm = pixel_norm(z)

x = tf.expand_dims(z_norm, axis=2)

x = tf.pad(x, [[0, 0], [0, 0], [15, 15]])
print x
print g1n_to_var['G1a.W'].shape

x = tf.nn.conv1d(x, g1n_to_var['G1a.W'], 1, padding='SAME', data_format='NCHW')
print x
"""
with tf.Session() as sess:
    print sess.run(x, {z: snapshot_z})
"""

Tensor("Pad:0", shape=(?, 512, 31), dtype=float32)
(512, 512, 16)
Tensor("conv1d/Squeeze:0", shape=(?, 16, 31), dtype=float32)


'\nwith tf.Session() as sess:\n    print sess.run(x, {z: snapshot_z})\n'